In [1]:
import pandas as pd
to_df_form = pd.read_csv("/home/kevin/mimic/icu_data_nan.csv",)
numeric_df = pd.read_csv("/home/kevin/mimic/icu_data.csv",).drop(columns=["subject_id", 'Weight', 'Height'])
print(numeric_df.columns)
print(len(to_df_form), len(numeric_df))
dod_counts = to_df_form["dod"].value_counts()[1]
sample_index = to_df_form[to_df_form["dod"]==0].sample(dod_counts).index
dod_index = to_df_form[to_df_form["dod"]==1].index
to_df_form.loc[to_df_form["time_interval"] < 0, "time_interval"] = 0
# numeric_df = pd.concat([numeric_df.loc[sample_index, :], numeric_df.loc[dod_index, :]],)
# to_df_form = to_df_form.loc[numeric_df.index, :]

Index(['pH', 'pO2', 'pCO2', 'HCO3', 'Glucose', 'K', 'Na+', 'Lactate', 'Hct',
       'SpO2', 'RR', 'HR', 'Temp', 'BP-mean', 'WBC', 'Albumin', 'Bilirubin',
       'PLT', 'BUN', 'Cr', 'HR signal', 'FiO2', 'UO', 'BMI', 'GCS score'],
      dtype='object')
50920 50920


In [2]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN, KMeansSMOTE, BorderlineSMOTE, SVMSMOTE

for i,v in numeric_df.isna().any().items():
    if v:
        numeric_df = numeric_df.drop(columns=[i])
dataset = numeric_df
dataset["dod"] = to_df_form["dod"]
dataset["time_interval"] = to_df_form["time_interval"]
dataset["Gender"] = to_df_form["Gender"]

train, test = train_test_split(dataset, test_size=0.1)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

# oversample = RandomOverSampler(sampling_strategy='minority')
oversample = ADASYN(sampling_strategy=0.75, n_neighbors=35)
train, train_dod = oversample.fit_resample(train.drop(columns=["dod", "Gender"]), train["dod"])
train["dod"] = train_dod
train["Gender"] = "X"

X = train.drop(columns=["dod", "time_interval", "Gender",]).values.astype(float)
y = train["dod"].values.reshape(-1,1).astype(int)
y_reg = train["time_interval"].values.reshape(-1,1).astype(int)

female_index = test[test["Gender"]=="F"].index
male_index = test[test["Gender"]=="M"].index

X_test = test.drop(columns=["dod", "Gender", "time_interval"]).values.astype(float)
y_test = test["dod"].values.reshape(-1,1).astype(int)
y_reg_test = test["time_interval"].values.reshape(-1,1).astype(int)

In [3]:
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import plotly.express as px

def scoring(X_val, y_val, y_reg_val, clf=None, reg=None, title="Test Dataset score"):
      y_pred = clf.predict(X_val,)
      y_reg_pred = reg.predict(X_val)
      print(title)
      print(classification_report(y_val, y_pred, labels=[0,1]))
      print("AUC ROC: ",roc_auc_score(y_val, y_pred))
      print("\nRMSE:", mean_squared_error(y_reg_val, y_reg_pred, squared=False),
            "\nMAE: ", mean_absolute_error(y_reg_val, y_reg_pred,),
            "\nR2: ", r2_score(y_reg_val, y_reg_pred,))
      y_pred = clf.predict_proba(X_val,)[:,1]
      fpr, tpr, thresholds = roc_curve(
            y_val,
            y_pred,)
      fig = px.area(
            x=fpr, y=tpr,
            title=f'ROC Curve (AUC={roc_auc_score(y_val, y_pred):.4f})',
            labels=dict(x='False Positive Rate', y='True Positive Rate'),
            width=700, height=500
      )
      fig.add_shape(
            type='line', line=dict(dash='dash'),
            x0=0, x1=1, y0=0, y1=1
      )

      fig.update_yaxes(scaleanchor="x", scaleratio=1)
      fig.update_xaxes(constrain='domain')
      fig.show()

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge, LogisticRegression, BayesianRidge
from sklearn.svm import SVC, LinearSVR, LinearSVC
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score, roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA 

scaler = StandardScaler() # MinMaxScaler()
X_ = scaler.fit_transform(X, y)
pca = PCA(n_components=7)
X_ = pca.fit_transform(X_)

poly = PolynomialFeatures(degree=2)
X_ = poly.fit_transform(X_, y_reg)

skf = StratifiedKFold(n_splits=3, shuffle=True, )
for train_idx, val_idx in skf.split(X, y):
    clf = SVC(class_weight={0:0.9, 1:1}, probability=True)
    reg = ElasticNet()
    X_train, X_reg_train, y_train, y_reg_train = X_[train_idx], X_[train_idx], y[train_idx], y_reg[train_idx]
    X_val, X_reg_val, y_val, y_reg_val = X_[val_idx], X_[val_idx], y[val_idx], y_reg[val_idx]
    clf.fit(X_train, y_train)
    reg.fit(X_reg_train, y_reg_train)
    y_pred = clf.predict(X_val,)
    y_reg_pred = reg.predict(X_reg_val)
    print(classification_report(y_val, y_pred, target_names=["alive", "dead"]))
    print("ROC", roc_auc_score(y_val, clf.predict_proba(X_val)[:,1]))
    print(mean_squared_error(y_reg_val, y_reg_pred, squared=False), r2_score(y_reg_val, y_reg_pred,))
    break


In [54]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score, roc_auc_score

scaler = MinMaxScaler() # StandardScaler() # 
X_ = scaler.fit_transform(X, y)
# pca = PCA(n_components=2)
# X_ = pca.fit_transform(X_)

skf = StratifiedKFold(n_splits=3, shuffle=True,)
for train_idx, val_idx in skf.split(X, y):
    clf = ExtraTreesClassifier(n_estimators=750, class_weight={0:1, 1:1.15}, criterion="entropy")
    reg= ExtraTreesRegressor(n_estimators=750,)
    X_train, X_reg_train, y_train, y_reg_train = X[train_idx], X_[train_idx], y[train_idx], y_reg[train_idx]
    X_val, X_reg_val, y_val, y_reg_val = X[val_idx], X_[val_idx], y[val_idx], y_reg[val_idx]
    clf.fit(X_train, y_train)
    reg.fit(X_reg_train, y_reg_train)
    y_pred = clf.predict(X_val,)
    y_reg_pred = reg.predict(X_reg_val)
    print(classification_report(y_val, y_pred, target_names=["alive", "dead"]))
    print("ROC", roc_auc_score(y_val, clf.predict_proba(X_val)[:,1]))
    print(mean_squared_error(y_reg_val, y_reg_pred, squared=False), r2_score(y_reg_val, y_reg_pred,))
    break


/tmp/ipykernel_6022/681770969.py:16: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/tmp/ipykernel_6022/681770969.py:17: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



              precision    recall  f1-score   support

       alive       0.82      0.82      0.82     10327
        dead       0.78      0.78      0.78      8311

    accuracy                           0.80     18638
   macro avg       0.80      0.80      0.80     18638
weighted avg       0.80      0.80      0.80     18638

ROC 0.8875143766236673
8.127904449919928 0.14042600389201743


In [37]:
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRFClassifier, XGBRFRegressor
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score, roc_auc_score

scaler = StandardScaler() # MinMaxScaler()
X_ = scaler.fit_transform(X, y)
pca = PCA(n_components=7)
X_ = pca.fit_transform(X_)

skf = StratifiedKFold(n_splits=3, shuffle=True,)
for train_idx, val_idx in skf.split(X, y):
    clf = XGBRFClassifier(n_estimators=1000, gamma=0.001, subsample=0.9, max_delta_step=3)
    reg= XGBRFRegressor(n_estimators=1000, gamma=0.001, subsample=0.9,)
    X_train, X_reg_train, y_train, y_reg_train = X_[train_idx], X_[train_idx], y[train_idx], y_reg[train_idx]
    X_val, X_reg_val, y_val, y_reg_val = X_[val_idx], X_[val_idx], y[val_idx], y_reg[val_idx]
    clf.fit(X_train, y_train)
    reg.fit(X_reg_train, y_reg_train)
    y_pred = clf.predict(X_val,)
    y_reg_pred = reg.predict(X_reg_val)
    print(classification_report(y_val, y_pred, target_names=["alive", "dead"]))
    print("ROC", roc_auc_score(y_val, clf.predict_proba(X_val)[:,1]))
    print(mean_squared_error(y_reg_val, y_reg_pred, squared=False), r2_score(y_reg_val, y_reg_pred,))
    break

              precision    recall  f1-score   support

       alive       0.67      0.71      0.69     10322
        dead       0.69      0.65      0.67     10323

    accuracy                           0.68     20645
   macro avg       0.68      0.68      0.68     20645
weighted avg       0.68      0.68      0.68     20645

ROC 0.7396671740337947
9.166380866964852 0.08750037864466476


In [38]:
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score, roc_auc_score

scaler = StandardScaler() # MinMaxScaler()
X_ = scaler.fit_transform(X, y)
pca = PCA(n_components=3)
X_ = pca.fit_transform(X_)

skf = StratifiedKFold(n_splits=3, shuffle=True, )
for train_idx, val_idx in skf.split(X, y):
    clf = CatBoostClassifier(verbose=0, iterations=1000, class_weights={0: 1, 1: 1})
    reg = CatBoostRegressor(verbose=0, iterations=1000,)
    X_train, X_reg_train, y_train, y_reg_train = X_[train_idx], X_[train_idx], y[train_idx], y_reg[train_idx]
    X_val, X_reg_val, y_val, y_reg_val = X_[val_idx], X_[val_idx], y[val_idx], y_reg[val_idx]
    clf.fit(X_train, y_train)
    reg.fit(X_reg_train, y_reg_train)
    y_pred = clf.predict(X_val,)
    y_reg_pred = reg.predict(X_reg_val)
    print(classification_report(y_val, y_pred, target_names=["alive", "dead"]))
    print("ROC", roc_auc_score(y_val, clf.predict_proba(X_val)[:,1]))
    print(mean_squared_error(y_reg_val, y_reg_pred, squared=False), r2_score(y_reg_val, y_reg_pred,))
    break


              precision    recall  f1-score   support

       alive       0.66      0.66      0.66     10322
        dead       0.66      0.66      0.66     10323

    accuracy                           0.66     20645
   macro avg       0.66      0.66      0.66     20645
weighted avg       0.66      0.66      0.66     20645

ROC 0.7239218251447065
8.746822909122884 0.11559000043983558


In [56]:
from xgboost import XGBClassifier, XGBRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor, VotingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score, roc_auc_score

scaler = MinMaxScaler() # StandardScaler() # 
X_ = scaler.fit_transform(X, y)
# pca = PCA(n_components=2)
# X_ = pca.fit_transform(X_)

skf = StratifiedKFold(n_splits=3, shuffle=True, )
for train_idx, val_idx in skf.split(X, y):
    clf = VotingClassifier([
       ("CatB", CatBoostClassifier(verbose=0, class_weights={0:1, 1:1.2})),
       ("ET", ExtraTreesClassifier(n_estimators=1000,class_weight={0:1, 1:1.2})),
      #  ("RF", RandomForestClassifier(n_estimators=1000,class_weight={0:1, 1:1.2}))
       ], voting="soft")
    reg = VotingRegressor([
       ("CatB", CatBoostRegressor(verbose=0,)),
       ("ET", ExtraTreesRegressor()),
      #  ("RF", RandomForestRegressor())
       ])
    X_train, X_reg_train, y_train, y_reg_train = X_[train_idx], X_[train_idx], y[train_idx], y_reg[train_idx]
    X_val, X_reg_val, y_val, y_reg_val = X_[val_idx], X_[val_idx], y[val_idx], y_reg[val_idx]
    clf.fit(X_train, y_train)
    reg.fit(X_reg_train, y_reg_train)
    y_pred = clf.predict(X_val,)
    y_reg_pred = reg.predict(X_reg_val)
    print(classification_report(y_val, y_pred, target_names=["alive", "dead"]))
    print("ROC:", roc_auc_score(y_val, y_pred))
    print("RMSE:", mean_squared_error(y_reg_val, y_reg_pred, squared=False),
          "MAE:", mean_absolute_error(y_reg_val, y_reg_pred,),  
          "R2:", r2_score(y_reg_val, y_reg_pred,))


/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



              precision    recall  f1-score   support

       alive       0.83      0.84      0.83     10327
        dead       0.80      0.79      0.79      8311

    accuracy                           0.82     18638
   macro avg       0.81      0.81      0.81     18638
weighted avg       0.81      0.82      0.81     18638

ROC: 0.8122036875811778
RMSE: 8.373017207955375 MAE: 4.872063258710204 R2: 0.14161677221083124


/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



              precision    recall  f1-score   support

       alive       0.83      0.84      0.84     10327
        dead       0.80      0.79      0.79      8310

    accuracy                           0.82     18637
   macro avg       0.82      0.81      0.81     18637
weighted avg       0.82      0.82      0.82     18637

ROC: 0.8144536706263545
RMSE: 7.885802362862999 MAE: 4.7674723341698435 R2: 0.15427401902851634


/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/ensemble/_voting.py:597: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



              precision    recall  f1-score   support

       alive       0.83      0.84      0.84     10327
        dead       0.80      0.79      0.79      8310

    accuracy                           0.82     18637
   macro avg       0.81      0.81      0.81     18637
weighted avg       0.82      0.82      0.82     18637

ROC: 0.8137772457953443
RMSE: 8.309198813853666 MAE: 4.863249155553503 R2: 0.1507357968720887


In [57]:
scoring(X_val=X_val, y_val=y_val, y_reg_val=y_reg_val, clf=clf, reg=reg, title="Valid Dataset score")

Valid Dataset score
              precision    recall  f1-score   support

           0       0.83      0.84      0.84     10327
           1       0.80      0.79      0.79      8310

    accuracy                           0.82     18637
   macro avg       0.81      0.81      0.81     18637
weighted avg       0.82      0.82      0.82     18637

AUC ROC:  0.8137772457953443

RMSE: 8.309198813853666 
MAE:  4.863249155553503 
R2:  0.1507357968720887


In [79]:
import pickle

with open("docker_img/python311/model", "wb") as f:
    pickle.dump([clf,reg],f)

In [ ]:
import shap
import matplotlib.pyplot as plt
shap.initjs()
plt.style.use('seaborn')
explainer = shap.TreeExplainer(clf,)
shap_values = explainer.shap_values(X_test, y_test)
shap.summary_plot(shap_values, X_test, feature_names=numeric_df.columns.tolist())
shap.force_plot(explainer.expected_value, shap_values[:200], feature_names=numeric_df.columns.values.reshape(1,-1).tolist())


In [58]:
scoring(X_val=X_test, y_val=y_test, y_reg_val=y_reg_test, clf=clf, reg=reg, title="Test Dataset score")
scoring(X_val=X_test[male_index], y_val=y_test[male_index], y_reg_val=y_reg_test[male_index], clf=clf, reg=reg, title="Male Only Test Dataset score")
scoring(X_val=X_test[female_index], y_val=y_test[female_index], y_reg_val=y_reg_test[female_index], clf=clf, reg=reg, title="Female Only Test Dataset score")


Test Dataset score
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3462
           1       0.32      1.00      0.48      1630

    accuracy                           0.32      5092
   macro avg       0.16      0.50      0.24      5092
weighted avg       0.10      0.32      0.16      5092

AUC ROC:  0.5

RMSE: 30.86922675210675 
MAE:  30.066179670204356 
R2:  -11.452741071117465


/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Male Only Test Dataset score
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1972
           1       0.31      1.00      0.47       876

    accuracy                           0.31      2848
   macro avg       0.15      0.50      0.24      2848
weighted avg       0.09      0.31      0.14      2848

AUC ROC:  0.5

RMSE: 30.883731443657002 
MAE:  30.074852451686567 
R2:  -10.60585568829094


/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



Female Only Test Dataset score
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1490
           1       0.34      1.00      0.50       754

    accuracy                           0.34      2244
   macro avg       0.17      0.50      0.25      2244
weighted avg       0.11      0.34      0.17      2244

AUC ROC:  0.5

RMSE: 30.85080812590911 
MAE:  30.055172503688606 
R2:  -12.74131741998508


/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/root/anaconda3/envs/torch_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [7]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.metrics import classification_report, mean_absolute_error, mean_squared_error, r2_score, roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
import os
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

scaler = MinMaxScaler()
X_ = scaler.fit_transform(X, y)
category_index = [20]
skf = StratifiedKFold(n_splits=3, shuffle=True, )
for train_idx, val_idx in skf.split(X, y):
    clf = TabNetClassifier(#cat_idxs=category_index, cat_dims=[125],
                           n_d=8, n_a=8, n_steps=7, gamma=1.95, 
                           n_independent=1, n_shared=1, output_dim=1,
                           optimizer_params=dict(lr=1e-2), verbose=0,
                           device_name="cuda")
    reg = TabNetRegressor(#cat_idxs=category_index, cat_dims=[125],
                           n_d=8, n_a=8, n_steps=7, gamma=1.95, 
                          n_independent=1, n_shared=1, output_dim=1,
                          optimizer_params=dict(lr=1e-2), verbose=0,
                          device_name="cuda")
    X_train, y_train, y_reg_train = X_[train_idx], y[train_idx].reshape(-1), y_reg[train_idx]
    X_val, y_val, y_reg_val = X_[val_idx], y[val_idx].reshape(-1), y_reg[val_idx]
    clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], patience=50, max_epochs=500,)
    reg.fit(X_train, y_reg_train, eval_set=[(X_val, y_reg_val)], patience=50, max_epochs=500, eval_metric=["rmse"])
    y_pred = clf.predict(X_val,)
    y_reg_pred = reg.predict(X_val)
    print(classification_report(y_val, y_pred, labels=[0, 1]))
    print("ROC:", roc_auc_score(y_val, y_pred))
    print("RMSE:", mean_squared_error(y_reg_val, y_reg_pred, squared=False),
          "MAE:", mean_absolute_error(y_reg_val, y_reg_pred),
          "R2:", r2_score(y_reg_val, y_reg_pred))


In [6]:
scoring(X_val=X_test, y_val=y_test, y_reg_val=y_reg_test, clf=clf, reg=reg, title="Test Dataset score")
scoring(X_val=X_test[male_index], y_val=y_test[male_index], y_reg_val=y_reg_test[male_index], clf=clf, reg=reg, title="Male Only Test Dataset score")
scoring(X_val=X_test[female_index], y_val=y_test[female_index], y_reg_val=y_reg_test[female_index], clf=clf, reg=reg, title="Female Only Test Dataset score")

Test Dataset score
              precision    recall  f1-score   support

           0       0.75      0.49      0.59      3417
           1       0.39      0.67      0.49      1675

    accuracy                           0.55      5092
   macro avg       0.57      0.58      0.54      5092
weighted avg       0.63      0.55      0.56      5092

AUC ROC:  0.5785308750365818

RMSE: 712.2491104855435 
MAE:  671.6171547903944 
R2:  -6364.830172236341


Male Only Test Dataset score
              precision    recall  f1-score   support

           0       0.76      0.45      0.56      1940
           1       0.36      0.68      0.47       880

    accuracy                           0.52      2820
   macro avg       0.56      0.57      0.52      2820
weighted avg       0.63      0.52      0.53      2820

AUC ROC:  0.565961808809747

RMSE: 715.7466011691497 
MAE:  677.1853095575427 
R2:  -5737.617581259461


Female Only Test Dataset score
              precision    recall  f1-score   support

           0       0.74      0.54      0.63      1477
           1       0.43      0.65      0.52       795

    accuracy                           0.58      2272
   macro avg       0.59      0.60      0.57      2272
weighted avg       0.63      0.58      0.59      2272

AUC ROC:  0.5967016261928181

RMSE: 707.8840004329436 
MAE:  664.7059767783528 
R2:  -7406.787258337702
